## Aggregates models & data to county / fip level

### Packages

In [4]:
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import xagg as xa

### inputs

In [14]:
filein = "/storage/group/pches/default/users/dcl5300/wbm_soilM_uc_2024_DATA/projections/eCONUS/out/LOCA2/GFDL-ESM4_r1i1p1f1_ssp245_MOSAIC_mse.nc"
county_shp_path = "/storage/work/cta5244/avila_et_al_2026_pyWBM_yield/shape_files/counties_contig_plot.shp"
ds = xr.open_dataset(filein)

us_county = gpd.read_file(county_shp_path)
us_county = us_county.set_crs("EPSG:4269", inplace=False)
us_county = us_county.to_crs("EPSG:4326")

# Area weight and aggregate
weightmap = xa.pixel_overlaps(ds, us_county)
aggregated = xa.aggregate(ds, weightmap)

# Convert to pandas
ds_out = aggregated.to_dataset().to_dataframe()
ds_out = ds_out.reset_index().set_index(['fips','time']).drop(columns='pix_idx')


creating polygons for each pixel...
lat/lon bounds not found in dataset; they will be created.
calculating overlaps between pixels and output polygons...


NoOverlapError: No `ds` grid cells overlapped with any polygon in `gdf_in`. Check the input `ds` and `gdf_in`.